In [ ]:
!pip install tf2onnx
!pip install onnx
!pip install onnxscript
!pip install onnxruntime-gpu

## Import Libraries

In [6]:
import time
import onnxruntime as ort
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
import tf2onnx
import onnx
import numpy as np

## ResNet Model

In [7]:
resnet_model = ResNet50(weights="imagenet", include_top=False)

## Convert TensorFlow Model To ONNX

In [8]:
input_shape = [tf.TensorSpec(resnet_model.inputs[0].shape, dtype=resnet_model.inputs[0].dtype, name=resnet_model.inputs[0].name)]
input_data = np.random.randn(1, 224, 224, 3).astype(np.float32)
onnx_model, _ = tf2onnx.convert.from_keras(resnet_model, input_signature=input_shape)
onnx.save(onnx_model, "tensorflow_to_onnx_model.onnx")

## Use ONNX Model Base CPU

In [5]:
onnx_model_session = ort.InferenceSession(
    "tensorflow_to_onnx_model.onnx",
    providers = ["CPUExecutionProvider"]
)
input_name = onnx_model_session.get_inputs()[0].name
output_name = onnx_model_session.get_outputs()[0].name
start_time = time.time()
predict = onnx_model_session.run(
    [output_name],
    {input_name: input_data}
)
end_time = time.time()
print(f"Inference Time: {end_time - start_time}")

Inference Time: 0.1308915615081787


## Use ONNX Model Base CUDA

In [9]:
session_options = ort.SessionOptions()
cuda = {'device_id': 0}

onnx_model_session = ort.InferenceSession(
    "tensorflow_to_onnx_model.onnx",
    sess_options=session_options,
    providers = [("CUDAExecutionProvider", cuda)]
)
input_name = onnx_model_session.get_inputs()[0].name
output_name = onnx_model_session.get_outputs()[0].name
start_time = time.time()
predict = onnx_model_session.run(
    [output_name],
    {input_name: input_data}
)
end_time = time.time()
print(f"Inference Time: {end_time - start_time}")

Inference Time: 0.07973051071166992
